# Demo 11: Performance Profiler

This notebook demonstrates the **Performance Profiler** module for identifying bottlenecks and optimizing holographic algorithms.

## Features

- Profile individual functions
- Profile algorithm components
- Analyze iterative algorithms
- Measure batch processing scaling
- Identify bottlenecks
- Compare before/after optimization

In [ ]:
import sys
sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt
from workbench import (
    PerformanceProfiler,
    profile,
    compare_profiles,
    estimate_complexity,
    format_time,
    format_memory,
    zetazero,
    SpectralScorer,
    holographic_refinement
)

print("✓ Imports successful")

## 1. Basic Function Profiling

Profile individual functions to measure execution time and memory usage.

In [ ]:
# Create profiler
profiler = PerformanceProfiler(track_memory=True, warmup_runs=2)

# Define test functions
def slow_computation(n):
    """Slow O(n) computation."""
    result = 0
    for i in range(n):
        result += np.sin(i) * np.cos(i)
    return result

def fast_computation(n):
    """Fast vectorized computation."""
    x = np.arange(n)
    return np.sum(np.sin(x) * np.cos(x))

# Profile both
result1, profile1 = profiler.profile_function(slow_computation, 10000, name="slow")
result2, profile2 = profiler.profile_function(fast_computation, 10000, name="fast")

print(f"Slow: {format_time(profile1.execution_time)}")
print(f"Fast: {format_time(profile2.execution_time)}")

# Compare
comparison = compare_profiles(profile1, profile2)
print(f"\nSpeedup: {comparison['speedup']:.2f}x")
print(f"Time improvement: {comparison['time_improvement_percent']:.1f}%")

## 2. Component Profiling

Profile multiple components of a holographic algorithm.

In [ ]:
profiler2 = PerformanceProfiler()

# Simulate holographic workflow components
def generate_signal(n):
    return np.random.randn(n) + 1j * np.random.randn(n)

def compute_fft(signal):
    return np.fft.fft(signal)

def extract_magnitude(fft_result):
    return np.abs(fft_result)

def smooth_result(magnitude):
    from scipy.ndimage import gaussian_filter1d
    return gaussian_filter1d(magnitude, sigma=5)

# Define components
n = 10000
components = {
    "signal_generation": (generate_signal, (n,), {}),
    "fft_computation": (compute_fft, (generate_signal(n),), {}),
    "magnitude_extraction": (extract_magnitude, (compute_fft(generate_signal(n)),), {}),
    "smoothing": (smooth_result, (np.abs(compute_fft(generate_signal(n))),), {}),
}

# Profile all components
results = profiler2.profile_components(components)

# Generate report
print(profiler2.generate_report())

# Identify bottlenecks
bottlenecks = profiler2.identify_bottlenecks(threshold=0.15)
print("\nBottleneck Analysis:")
for rec in bottlenecks.recommendations:
    print(f"  {rec}")

## 3. Iteration Profiling

Profile iterative algorithms and detect convergence.

In [ ]:
profiler3 = PerformanceProfiler()

# Simulate iterative refinement
def refinement_iteration(x, target=0.0):
    """One iteration of gradient descent."""
    return x - 0.1 * (x - target)

# Profile iterations
iter_profile = profiler3.profile_iterations(
    refinement_iteration,
    iterations=20,
    x=10.0,
    target=0.0,
    convergence_threshold=1e-5
)

print(f"Total iterations: {iter_profile.total_iterations}")
print(f"Total time: {format_time(iter_profile.total_time)}")
print(f"Avg time per iteration: {format_time(iter_profile.avg_time_per_iteration)}")
print(f"Std time per iteration: {format_time(iter_profile.std_time_per_iteration)}")

if iter_profile.convergence_detected:
    print(f"\n✓ Convergence detected at iteration {iter_profile.convergence_iteration}")
else:
    print("\n✗ No convergence detected")

# Plot iteration times
plt.figure(figsize=(10, 4))
plt.plot(iter_profile.iteration_times, 'o-')
plt.xlabel('Iteration')
plt.ylabel('Time (s)')
plt.title('Per-Iteration Execution Time')
plt.grid(True, alpha=0.3)
plt.show()

## 4. Batch Profiling

Analyze how performance scales with batch size.

In [ ]:
profiler4 = PerformanceProfiler()

# Batch processing function
def process_batch(data, batch_size=100):
    """Process data in batches."""
    results = []
    for i in range(0, len(data), batch_size):
        batch = data[i:i+batch_size]
        results.append(np.mean(batch))
    return np.array(results)

# Test data
data = np.random.randn(100000)

# Profile different batch sizes
batch_profile = profiler4.profile_batch(
    process_batch,
    batch_sizes=[10, 50, 100, 500, 1000, 5000],
    data=data
)

print(f"Scaling factor: {batch_profile.scaling_factor:.2f}")
print(f"Estimated complexity: {estimate_complexity(batch_profile)}")
print(f"Optimal batch size: {batch_profile.optimal_batch_size}")

# Plot scaling
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 4))

# Total time vs batch size
ax1.plot(batch_profile.batch_sizes, batch_profile.batch_times, 'o-')
ax1.set_xlabel('Batch Size')
ax1.set_ylabel('Total Time (s)')
ax1.set_title('Total Processing Time')
ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.grid(True, alpha=0.3)

# Time per item vs batch size
ax2.plot(batch_profile.batch_sizes, batch_profile.times_per_item, 'o-', color='orange')
ax2.set_xlabel('Batch Size')
ax2.set_ylabel('Time per Item (s)')
ax2.set_title('Efficiency (Lower is Better)')
ax2.set_xscale('log')
ax2.set_yscale('log')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Context Manager Profiling

Profile code blocks using context managers.

In [ ]:
profiler5 = PerformanceProfiler()

# Profile different stages
with profiler5.profile_block("data_preparation"):
    data = np.random.randn(50000)
    data_normalized = (data - np.mean(data)) / np.std(data)

with profiler5.profile_block("fft_analysis"):
    fft_result = np.fft.fft(data_normalized)
    magnitude = np.abs(fft_result)

with profiler5.profile_block("peak_detection"):
    threshold = np.mean(magnitude) + 2 * np.std(magnitude)
    peaks = np.where(magnitude > threshold)[0]

with profiler5.profile_block("visualization_prep"):
    sorted_peaks = peaks[np.argsort(magnitude[peaks])[::-1]]
    top_peaks = sorted_peaks[:10]

print(profiler5.generate_report())

## 6. Real-World Example: Profiling Zeta Zero Computation

Profile the `zetazero` function from the workbench.

In [ ]:
profiler6 = PerformanceProfiler()

# Profile zeta zero computation for different indices
indices = [10, 50, 100, 500, 1000]

for n in indices:
    _, profile = profiler6.profile_function(zetazero, n, name=f"zetazero({n})")
    print(f"zetazero({n:4d}): {format_time(profile.execution_time)}")

print("\n" + profiler6.generate_report())

## 7. Profiling Holographic Refinement

Profile the complete holographic refinement workflow.

In [ ]:
profiler7 = PerformanceProfiler()

# Generate test signals
n = 5000
object_signal = np.random.randn(n) + 0.5 * np.sin(np.linspace(0, 10*np.pi, n))
reference_signal = np.sin(np.linspace(0, 10*np.pi, n))

# Profile holographic refinement
_, profile = profiler7.profile_function(
    holographic_refinement,
    object_signal,
    reference_signal,
    name="holographic_refinement",
    method="hilbert",
    blend_ratio=0.6
)

print(f"Holographic refinement: {format_time(profile.execution_time)}")
print(f"Memory used: {format_memory(profile.memory_delta)}")

## 8. Decorator Pattern

Use the `@profile` decorator for automatic profiling.

In [ ]:
profiler8 = PerformanceProfiler()

@profile(name="matrix_multiply", profiler=profiler8)
def matrix_multiply(n):
    """Multiply two random matrices."""
    A = np.random.randn(n, n)
    B = np.random.randn(n, n)
    return A @ B

# Call decorated function
for size in [100, 200, 300]:
    result = matrix_multiply(size)
    print(f"Matrix {size}x{size} completed")

print("\n" + profiler8.generate_report())

## 9. Optimization Workflow

Complete workflow: profile → identify bottlenecks → optimize → compare.

In [ ]:
# Original (slow) implementation
def slow_spectral_analysis(signal):
    """Slow spectral analysis."""
    result = []
    for i in range(len(signal)):
        result.append(np.abs(signal[i]))
    return np.array(result)

# Optimized (fast) implementation
def fast_spectral_analysis(signal):
    """Fast vectorized spectral analysis."""
    return np.abs(signal)

# Profile both
profiler9 = PerformanceProfiler()
test_signal = np.random.randn(10000) + 1j * np.random.randn(10000)

_, before = profiler9.profile_function(slow_spectral_analysis, test_signal, name="before_opt")
_, after = profiler9.profile_function(fast_spectral_analysis, test_signal, name="after_opt")

# Compare
comparison = compare_profiles(before, after)

print("OPTIMIZATION RESULTS")
print("=" * 50)
print(f"Before: {format_time(before.execution_time)}")
print(f"After:  {format_time(after.execution_time)}")
print(f"\nSpeedup: {comparison['speedup']:.2f}x")
print(f"Time saved: {format_time(comparison['time_saved_seconds'])}")
print(f"Improvement: {comparison['time_improvement_percent']:.1f}%")

# Visualize
fig, ax = plt.subplots(figsize=(8, 5))
times = [before.execution_time, after.execution_time]
labels = ['Before\nOptimization', 'After\nOptimization']
colors = ['#e74c3c', '#2ecc71']

bars = ax.bar(labels, times, color=colors, alpha=0.7, edgecolor='black')
ax.set_ylabel('Execution Time (s)')
ax.set_title('Optimization Impact')
ax.grid(True, alpha=0.3, axis='y')

# Add value labels
for bar, time in zip(bars, times):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            format_time(time),
            ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

## Summary

The Performance Profiler provides:

1. **Function Profiling** - Measure individual function performance
2. **Component Analysis** - Profile algorithm components
3. **Iteration Tracking** - Monitor iterative algorithms
4. **Batch Scaling** - Analyze batch processing efficiency
5. **Bottleneck Detection** - Identify performance hotspots
6. **Memory Tracking** - Monitor memory usage
7. **Comparison Tools** - Compare before/after optimization
8. **Multiple Interfaces** - Functions, decorators, context managers

Use it to optimize any holographic algorithm in the workbench!